## Maven Hospital Challenge

#### Challenge Objective

For the Maven Hospital Challenge, you'll play the role of an Analytics Consultant for Massachusetts General Hospital (MGH).

You've been asked to build a high-level KPI report for the executive team, based on a subset of patient records. The purpose of the report is to give stakeholders visibility into the hospital's recent performance, and answer the following questions:

 - How many patients have been admitted or readmitted over time?
 - How long are patients staying in the hospital, on average?
 - How much is the average cost per visit?
 - How many procedures are covered by insurance?

The dashboard should scale to accommodate new data over time, but the CEO has asked you to summarize any insights you can derive from the sample provided.

#### About The Data Set
Synthetic data on ~1k patients of Massachussets General Hospital from 2011-2022, including information on patient demographics, insurance coverage, and medical encounters & procedures.

In [97]:
# import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# import warnings
# warnings.filterwarnings('ignore')

In [98]:
# Let's import all the datasets required for this task
patient_encounters = pd.read_csv("encounters.csv")
hospital_details = pd.read_csv("organizations.csv")
patient_demographic = pd.read_csv("patients.csv")
insurance_payer = pd.read_csv("payers.csv")
operating_procedures = pd.read_csv("procedures.csv")

In [99]:
# Let's check the number of features and records each dataset has
print("patient encounters data:", patient_encounters.shape)
print("hospital details:", hospital_details.shape)
print("patient demographic:", patient_demographic.shape)
print("insurance payer:", insurance_payer.shape)
print("operating procedures:", operating_procedures.shape)

patient encounters data: (27891, 14)
hospital details: (1, 8)
patient demographic: (974, 20)
insurance payer: (10, 7)
operating procedures: (47701, 9)


In [100]:
# Let's look at some few records from each dataset
print("patient encounters:")
patient_encounters.head(5)

patient encounters:


,Id,START,STOP,PATIENT,ORGANIZATION,PAYER,ENCOUNTERCLASS,CODE,DESCRIPTION,BASE_ENCOUNTER_COST,TOTAL_CLAIM_COST,PAYER_COVERAGE,REASONCODE,REASONDESCRIPTION
0,32c84703-2481-49cd-d571-3899d5820253,2011-01-02T09:26:36Z,2011-01-02T12:58:36Z,3de74169-7f67-9304-91d4-757e0f3a14d2,d78e84ec-30aa-3bba-a33a-f29a3a454662,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,ambulatory,185347001,Encounter for problem (procedure),85.55,1018.02,0.00,NaN,NaN
1,c98059da-320a-c0a6-fced-c8815f3e3f39,2011-01-03T05:44:39Z,2011-01-03T06:01:42Z,d9ec2e44-32e9-9148-179a-1653348cc4e2,d78e84ec-30aa-3bba-a33a-f29a3a454662,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,outpatient,308335008,Patient encounter procedure,142.58,2619.36,0.00,NaN,NaN
2,4ad28a3a-2479-782b-f29c-d5b3f41a001e,2011-01-03T14:32:11Z,2011-01-03T14:47:11Z,73babadf-5b2b-fee7-189e-6f41ff213e01,d78e84ec-30aa-3bba-a33a-f29a3a454662,7caa7254-5050-3b5e-9eae-bd5ea30e809c,outpatient,185349003,Encounter for check up (procedure),85.55,461.59,305.27,NaN,NaN
3,c3f4da61-e4b4-21d5-587a-fbc89943bc19,2011-01-03T16:24:45Z,2011-01-03T16:39:45Z,3b46a0b7-0f34-9b9a-c319-ace4a1f58c0b,d78e84ec-30aa-3bba-a33a-f29a3a454662,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,wellness,162673000,General examination of patient (procedure),136.80,1784.24,0.00,NaN,NaN
4,a9183b4f-2572-72ea-54c2-b3cd038b4be7,2011-01-03T17:36:53Z,2011-01-03T17:51:53Z,fa006887-d93c-d302-8b89-f3c25f88c0e1,d78e84ec-30aa-3bba-a33a-f29a3a454662,42c4fca7-f8a9-3cd1-982a-dd9751bf3e2a,ambulatory,390906007,Follow-up encounter,85.55,234.72,0.00,55822004.0,Hyperlipidemia


In [101]:
print("hospital details:") 
hospital_details.head(5)

hospital details:


,Id,NAME,ADDRESS,CITY,STATE,ZIP,LAT,LON
0,d78e84ec-30aa-3bba-a33a-f29a3a454662,MASSACHUSETTS GENERAL HOSPITAL,55 FRUIT STREET,BOSTON,MA,2114,42.362813,-71.069187


In [102]:
print("patient demographic:") 
patient_demographic.head(5)

patient demographic:


,Id,BIRTHDATE,DEATHDATE,PREFIX,FIRST,LAST,SUFFIX,MAIDEN,MARITAL,RACE,ETHNICITY,GENDER,BIRTHPLACE,ADDRESS,CITY,STATE,COUNTY,ZIP,LAT,LON
0,5605b66b-e92d-c16c-1b83-b8bf7040d51f,1977-03-19,NaN,Mrs.,Nikita578,Erdman779,NaN,Leannon79,M,white,nonhispanic,F,Wakefield Massachusetts US,510 Little Station Unit 69,Quincy,Massachusetts,Norfolk County,2186.0,42.290937,-70.975503
1,6e5ae27c-8038-7988-e2c0-25a103f01bfa,1940-02-19,NaN,Mr.,Zane918,Hodkiewicz467,NaN,NaN,M,white,nonhispanic,M,Brookline Massachusetts US,747 Conn Throughway,Boston,Massachusetts,Suffolk County,2135.0,42.308831,-71.063162
2,8123d076-0886-9007-e956-d5864aa121a7,1958-06-04,NaN,Mr.,Quinn173,Marquardt819,NaN,NaN,M,white,nonhispanic,M,Gardner Massachusetts US,816 Okuneva Extension Apt 91,Quincy,Massachusetts,Norfolk County,2170.0,42.265177,-70.967085
3,770518e4-6133-648e-60c9-071eb2f0e2ce,1928-12-25,2017-09-29,Mr.,Abel832,Smitham825,NaN,NaN,M,white,hispanic,M,Randolph Massachusetts US,127 Cole Way Unit 95,Boston,Massachusetts,Suffolk County,2118.0,42.334304,-71.066801
4,f96addf5-81b9-0aab-7855-d208d3d352c5,1928-12-25,2014-02-23,Mr.,Edwin773,Labadie908,NaN,NaN,M,white,hispanic,M,Stow Massachusetts US,976 Ziemann Gateway,Boston,Massachusetts,Suffolk County,2125.0,42.346771,-71.058813


In [103]:
print("insurance payer:")
insurance_payer.head(5)

insurance payer:


,Id,NAME,ADDRESS,CITY,STATE_HEADQUARTERED,ZIP,PHONE
0,b3221cfc-24fb-339e-823d-bc4136cbc4ed,Dual Eligible,7500 Security Blvd,Baltimore,MD,21244.0,1-877-267-2323
1,7caa7254-5050-3b5e-9eae-bd5ea30e809c,Medicare,7500 Security Blvd,Baltimore,MD,21244.0,1-800-633-4227
2,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,Medicaid,7500 Security Blvd,Baltimore,MD,21244.0,1-877-267-2323
3,d47b3510-2895-3b70-9897-342d681c769d,Humana,500 West Main St,Louisville,KY,40018.0,1-844-330-7799
4,6e2f1a2d-27bd-3701-8d08-dae202c58632,Blue Cross Blue Shield,Michigan Plaza,Chicago,IL,60007.0,1-800-262-2583


In [104]:
print("operating procedures:") 
operating_procedures.head(5)

operating procedures:


,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION,BASE_COST,REASONCODE,REASONDESCRIPTION
0,2011-01-02T09:26:36Z,2011-01-02T12:58:36Z,3de74169-7f67-9304-91d4-757e0f3a14d2,32c84703-2481-49cd-d571-3899d5820253,265764009,Renal dialysis (procedure),903,NaN,NaN
1,2011-01-03T05:44:39Z,2011-01-03T06:01:42Z,d9ec2e44-32e9-9148-179a-1653348cc4e2,c98059da-320a-c0a6-fced-c8815f3e3f39,76601001,Intramuscular injection,2477,NaN,NaN
2,2011-01-04T14:49:55Z,2011-01-04T15:04:55Z,d856d6e6-4c98-e7a2-129b-44076c63d008,2cfd4ddd-ad13-fe1e-528b-15051cea2ec3,703423002,Combined chemotherapy and radiation therapy (p...,11620,363406005.0,Malignant tumor of colon
3,2011-01-05T04:02:09Z,2011-01-05T04:17:09Z,bc9d59c3-0a30-6e3b-f47d-022e4f03c8de,17966936-0878-f4db-128b-a43ae10d0878,173160006,Diagnostic fiberoptic bronchoscopy (procedure),9796,162573006.0,Suspected lung cancer (situation)
4,2011-01-05T12:58:36Z,2011-01-05T16:42:36Z,3de74169-7f67-9304-91d4-757e0f3a14d2,9de5f0b0-4ba4-ce6f-45fb-b55c202f31a5,265764009,Renal dialysis (procedure),1255,NaN,NaN


In [105]:
# Let's check the datatypes present
print("patient encounters:")
patient_encounters.dtypes

patient encounters:


Id                      object
START                   object
STOP                    object
PATIENT                 object
ORGANIZATION            object
PAYER                   object
ENCOUNTERCLASS          object
CODE                     int64
DESCRIPTION             object
BASE_ENCOUNTER_COST    float64
TOTAL_CLAIM_COST       float64
PAYER_COVERAGE         float64
REASONCODE             float64
REASONDESCRIPTION       object
dtype: object

In [106]:
print("hospital details:") 
hospital_details.dtypes

hospital details:


Id          object
NAME        object
ADDRESS     object
CITY        object
STATE       object
ZIP          int64
LAT        float64
LON        float64
dtype: object

In [107]:
print("patient demographic:")
patient_demographic.dtypes

patient demographic:


Id             object
BIRTHDATE      object
DEATHDATE      object
PREFIX         object
FIRST          object
LAST           object
SUFFIX         object
MAIDEN         object
MARITAL        object
RACE           object
ETHNICITY      object
GENDER         object
BIRTHPLACE     object
ADDRESS        object
CITY           object
STATE          object
COUNTY         object
ZIP           float64
LAT           float64
LON           float64
dtype: object

In [108]:
print("insurance payer:")
insurance_payer.dtypes

insurance payer:


Id                      object
NAME                    object
ADDRESS                 object
CITY                    object
STATE_HEADQUARTERED     object
ZIP                    float64
PHONE                   object
dtype: object

In [109]:
print("operating procedures:") 
operating_procedures.dtypes

operating procedures:


START                 object
STOP                  object
PATIENT               object
ENCOUNTER             object
CODE                   int64
DESCRIPTION           object
BASE_COST              int64
REASONCODE           float64
REASONDESCRIPTION     object
dtype: object

In [110]:
# first we will convert some columns to proper datatypes
# patient_encounters
patient_encounters["START"] = pd.to_datetime(patient_encounters["START"], format='%Y-%m-%d %H:%M:%S')
patient_encounters["STOP"] = pd.to_datetime(patient_encounters["STOP"], format='%Y-%m-%d %H:%M:%S')
patient_encounters["CODE"] = patient_encounters["CODE"].astype("object")
patient_encounters["REASONCODE"] = patient_encounters["REASONCODE"].astype("object")

# hospital_details
hospital_details["ZIP"] = hospital_details["ZIP"].astype("object")
hospital_details["LAT"] = hospital_details["LAT"].astype("object")
hospital_details["LON"] = hospital_details["LON"].astype("object")

# patient_demographic
patient_demographic["BIRTHDATE"] = pd.to_datetime(patient_demographic["BIRTHDATE"], format='%Y-%m-%d')
patient_demographic["DEATHDATE"] = pd.to_datetime(patient_demographic["DEATHDATE"], format='%Y-%m-%d')
patient_demographic["ZIP"] = patient_demographic["ZIP"].astype("object")
patient_demographic["LAT"] = patient_demographic["LAT"].astype("object")
patient_demographic["LON"] = patient_demographic["LON"].astype("object")

# insurance_payer
insurance_payer["ZIP"] = insurance_payer["ZIP"].astype("object")

# operating_procedures
operating_procedures["START"] = pd.to_datetime(operating_procedures["START"], format='%Y-%m-%d %H:%M:%S')
operating_procedures["STOP"] = pd.to_datetime(operating_procedures["STOP"], format='%Y-%m-%d %H:%M:%S')
operating_procedures["CODE"] = operating_procedures["CODE"].astype("object")
operating_procedures["REASONCODE"] = operating_procedures["REASONCODE"].astype("object")

In [111]:
# Descriptive Statistics
print("patient encounters:")
patient_encounters.describe(include="all").T

patient encounters:


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_5160\1233353208.py:3: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  patient_encounters.describe(include="all").T
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_5160\1233353208.py:3: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  patient_encounters.describe(include="all").T


,count,unique,top,freq,first,last,mean,std,min,25%,50%,75%,max
Id,27891,27891,32c84703-2481-49cd-d571-3899d5820253,1,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
START,27891,27541,2016-12-08 10:00:40+00:00,3,2011-01-02 09:26:36+00:00,2022-02-05 20:27:36+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
STOP,27891,27765,2016-12-08 10:15:40+00:00,3,2011-01-02 12:58:36+00:00,2022-02-05 20:42:36+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PATIENT,27891,974,1712d26d-822d-1e3a-2267-0a9dba31d7c8,1381,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ORGANIZATION,27891,1,d78e84ec-30aa-3bba-a33a-f29a3a454662,27891,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PAYER,27891,10,7caa7254-5050-3b5e-9eae-bd5ea30e809c,11371,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ENCOUNTERCLASS,27891,6,ambulatory,12537,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CODE,27891.0,45.0,185347001.0,5261.0,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DESCRIPTION,27891,53,Encounter for problem (procedure),4308,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BASE_ENCOUNTER_COST,27891.0,NaN,NaN,NaN,NaT,NaT,116.181614,28.410082,85.55,85.55,136.8,142.58,146.18


In [112]:
print("patient demographic:") 
patient_demographic.describe(include="all").T

patient demographic:


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_5160\1543264315.py:2: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  patient_demographic.describe(include="all").T
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_5160\1543264315.py:2: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  patient_demographic.describe(include="all").T


,count,unique,top,freq,first,last
Id,974,974,5605b66b-e92d-c16c-1b83-b8bf7040d51f,1,NaT,NaT
BIRTHDATE,974,880,1925-11-17 00:00:00,4,1922-03-24,1991-11-27
DEATHDATE,154,148,2017-09-29 00:00:00,2,2011-02-03,2022-01-27
PREFIX,974,3,Mr.,494,NaT,NaT
FIRST,974,842,Domenic627,3,NaT,NaT
LAST,974,498,Heaney114,6,NaT,NaT
SUFFIX,21,3,PhD,10,NaT,NaT
MAIDEN,386,279,Jerde200,5,NaT,NaT
MARITAL,973,2,M,784,NaT,NaT
RACE,974,6,white,680,NaT,NaT


In [113]:
print("insurance payer:")
insurance_payer.describe(include="all").T

insurance payer:


,count,unique,top,freq
Id,10,10,b3221cfc-24fb-339e-823d-bc4136cbc4ed,1
NAME,10,10,Dual Eligible,1
ADDRESS,9,7,7500 Security Blvd,3
CITY,9,7,Baltimore,3
STATE_HEADQUARTERED,9,6,MD,3
ZIP,9.0,7.0,21244.0,3.0
PHONE,9,8,1-877-267-2323,2


In [114]:
print("operating procedures:") 
operating_procedures.describe(include="all").T

operating procedures:


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_5160\1155938861.py:2: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  operating_procedures.describe(include="all").T
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_5160\1155938861.py:2: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  operating_procedures.describe(include="all").T


,count,unique,top,freq,first,last,mean,std,min,25%,50%,75%,max
START,47701,39251,2013-09-30 22:31:23+00:00,21,2011-01-02 09:26:36+00:00,2022-01-29 20:35:37+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
STOP,47701,42263,2019-03-12 08:27:16+00:00,20,2011-01-02 12:58:36+00:00,2022-01-29 21:08:12+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PATIENT,47701,793,1712d26d-822d-1e3a-2267-0a9dba31d7c8,1783,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ENCOUNTER,47701,14670,66b2ab44-a2cc-8053-8f4e-c5be57e50cc4,186,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CODE,47701.0,157.0,710824005.0,4596.0,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DESCRIPTION,47701,163,Assessment of health and social care needs (pr...,4596,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BASE_COST,47701.0,NaN,NaN,NaN,NaT,NaT,2212.064967,5572.978748,1.0,431.0,431.0,966.0,289531.0
REASONCODE,10756.0,46.0,72892002.0,5718.0,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
REASONDESCRIPTION,10756,46,Normal pregnancy,5718,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [116]:
# Let's look at the concise information on each dataset
print("patient encounters:")
patient_encounters.info()

patient encounters:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27891 entries, 0 to 27890
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype              
---  ------               --------------  -----              
 0   Id                   27891 non-null  object             
 1   START                27891 non-null  datetime64[ns, UTC]
 2   STOP                 27891 non-null  datetime64[ns, UTC]
 3   PATIENT              27891 non-null  object             
 4   ORGANIZATION         27891 non-null  object             
 5   PAYER                27891 non-null  object             
 6   ENCOUNTERCLASS       27891 non-null  object             
 7   CODE                 27891 non-null  object             
 8   DESCRIPTION          27891 non-null  object             
 9   BASE_ENCOUNTER_COST  27891 non-null  float64            
 10  TOTAL_CLAIM_COST     27891 non-null  float64            
 11  PAYER_COVERAGE       27891 non-null  float64            
 12

In [117]:
print("patient demographic:") 
patient_demographic.info()

patient demographic:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 974 entries, 0 to 973
Data columns (total 20 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Id          974 non-null    object        
 1   BIRTHDATE   974 non-null    datetime64[ns]
 2   DEATHDATE   154 non-null    datetime64[ns]
 3   PREFIX      974 non-null    object        
 4   FIRST       974 non-null    object        
 5   LAST        974 non-null    object        
 6   SUFFIX      21 non-null     object        
 7   MAIDEN      386 non-null    object        
 8   MARITAL     973 non-null    object        
 9   RACE        974 non-null    object        
 10  ETHNICITY   974 non-null    object        
 11  GENDER      974 non-null    object        
 12  BIRTHPLACE  974 non-null    object        
 13  ADDRESS     974 non-null    object        
 14  CITY        974 non-null    object        
 15  STATE       974 non-null    object        
 16  COUNT

In [118]:
print("insurance payer:")
insurance_payer.info()

insurance payer:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Id                   10 non-null     object
 1   NAME                 10 non-null     object
 2   ADDRESS              9 non-null      object
 3   CITY                 9 non-null      object
 4   STATE_HEADQUARTERED  9 non-null      object
 5   ZIP                  9 non-null      object
 6   PHONE                9 non-null      object
dtypes: object(7)
memory usage: 688.0+ bytes


In [119]:
print("operating procedures:") 
operating_procedures.info()

operating procedures:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47701 entries, 0 to 47700
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   START              47701 non-null  datetime64[ns, UTC]
 1   STOP               47701 non-null  datetime64[ns, UTC]
 2   PATIENT            47701 non-null  object             
 3   ENCOUNTER          47701 non-null  object             
 4   CODE               47701 non-null  object             
 5   DESCRIPTION        47701 non-null  object             
 6   BASE_COST          47701 non-null  int64              
 7   REASONCODE         10756 non-null  object             
 8   REASONDESCRIPTION  10756 non-null  object             
dtypes: datetime64[ns, UTC](2), int64(1), object(6)
memory usage: 3.3+ MB
